In [2]:
!gdown https://drive.google.com/uc?id=1nvbP9_QjiQRtgstohXnkt-Bms_av6hd1

Downloading...
From (original): https://drive.google.com/uc?id=1nvbP9_QjiQRtgstohXnkt-Bms_av6hd1
From (redirected): https://drive.google.com/uc?id=1nvbP9_QjiQRtgstohXnkt-Bms_av6hd1&confirm=t&uuid=2f928783-dc2a-4701-abf7-d9ce0747aafb
To: /content/creditcard.csv
100% 151M/151M [00:01<00:00, 85.3MB/s]


In [4]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, BatchNormalization, Flatten, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve, precision_recall_curve, auc
)
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# Download dataset (if needed)
# !gdown https://drive.google.com/uc?id=1nvbP9_QjiQRtgstohXnkt-Bms_av6hd1

# Load dataset
df = pd.read_csv('creditcard.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Original class distribution:\n{df['Class'].value_counts()}")

# Step 1: Separate features and labels
X = df.drop('Class', axis=1).values  # 30 features
y = df['Class'].values

# Step 2: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: First split - separate test set (20% of total data)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_scaled, y, test_size=0.20, stratify=y, random_state=42
)

# Step 4: Second split - split remaining 80% into train (75% of 80% = 60% total)
# and validation (25% of 80% = 20% total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
)

print(f"\nBefore SMOTE:")
print(f"Training set: {X_train.shape[0]} samples - Fraud: {sum(y_train)}, Legit: {len(y_train) - sum(y_train)}")
print(f"Validation set: {X_val.shape[0]} samples - Fraud: {sum(y_val)}, Legit: {len(y_val) - sum(y_val)}")
print(f"Test set: {X_test.shape[0]} samples - Fraud: {sum(y_test)}, Legit: {len(y_test) - sum(y_test)}")

# Step 5: Apply SMOTE ONLY to training data
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"\nAfter SMOTE (Training set only):")
print(f"Training set: {X_train_balanced.shape[0]} samples - Fraud: {sum(y_train_balanced)}, Legit: {len(y_train_balanced) - sum(y_train_balanced)}")
print(f"Validation set: UNCHANGED (original imbalanced distribution)")
print(f"Test set: UNCHANGED (original imbalanced distribution)")

# Step 6: Calculate class weights to improve precision
print("\n" + "="*60)
print("CALCULATING CLASS WEIGHTS FOR PRECISION OPTIMIZATION")
print("="*60)

classes = np.unique(y_train_balanced)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train_balanced)

# Increase fraud class weight by 1.5x to reduce false positives
class_weight_dict = {
    0: class_weights[0],
    1: class_weights[1] * 1.5  # Emphasize fraud class
}

print(f"Original Class Weights: Class 0 = {class_weights[0]:.4f}, Class 1 = {class_weights[1]:.4f}")
print(f"Adjusted Class Weights: Class 0 = {class_weight_dict[0]:.4f}, Class 1 = {class_weight_dict[1]:.4f}")
print("Effect: Model will penalize false positives more heavily")


# Step 7: Reshape for Conv1D input (samples, features, channels)
X_train_reshaped = X_train_balanced.reshape((X_train_balanced.shape[0], 30, 1))
X_val_reshaped = X_val.reshape((X_val.shape[0], 30, 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], 30, 1))

# Step 8: Build the 14-Layer CNN
def build_14layer_cnn(input_shape):
    model = Sequential([
        # Layer 1: Conv1D
        Input(shape=input_shape),
        Conv1D(32, kernel_size=2, activation='relu', padding='valid'),

        # Layer 2: Batch Normalization
        BatchNormalization(),

        # Layer 3: Dropout
        Dropout(0.2),

        # Layer 4: Conv1D
        Conv1D(64, kernel_size=2, activation='relu', padding='valid'),

        # Layer 5: Batch Normalization
        BatchNormalization(),

        # Layer 6: Dropout
        Dropout(0.5),

        # Layer 7: Flatten
        Flatten(),

        # Layer 8: Dense
        Dense(64, activation='relu'),

        # Layer 9: Dropout
        Dropout(0.5),

        # Layer 10: Dense
        Dense(100, activation='relu'),

        # Layer 11: Dense
        Dense(50, activation='relu'),

        # Layer 12: Dense
        Dense(25, activation='relu'),

        # Layer 13-14: Output Dense
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

# Step 9: Create the model
model = build_14layer_cnn((30, 1))
model.summary()

# Step 10: Train for 100 epochs with class weights
print("\n" + "="*60)
print("STARTING TRAINING WITH CLASS WEIGHTS")
print("="*60)
train_start_time = time.time()

history = model.fit(
    X_train_reshaped, y_train_balanced,
    epochs=100,
    batch_size=2048,
    validation_data=(X_val_reshaped, y_val),
    class_weight=class_weight_dict,  # ADDED CLASS WEIGHTS
    verbose=1
)

train_end_time = time.time()
train_time = train_end_time - train_start_time
print(f"\nTraining completed in {train_time:.2f} seconds ({train_time/60:.2f} minutes)")

# Step 11: Save training history to CSV
history_df = pd.DataFrame(history.history)
history_df['epoch'] = range(1, 101)
history_df = history_df[['epoch', 'loss', 'accuracy', 'val_loss', 'val_accuracy']]
history_df.to_csv('14layer_cnn_optimized_training_history.csv', index=False)
print("\nTraining history saved to '14layer_cnn_optimized_training_history.csv'")

# Step 12: Evaluate on VALIDATION SET
print("\n" + "="*60)
print("VALIDATION SET PERFORMANCE (during training)")
print("="*60)
val_loss, val_accuracy = model.evaluate(X_val_reshaped, y_val, verbose=0)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Step 13: Get predictions on TEST SET
print("\n" + "="*60)
print("THRESHOLD OPTIMIZATION FOR PRECISION")
print("="*60)
test_start_time = time.time()
y_pred_proba = model.predict(X_test_reshaped)
test_end_time = time.time()
test_time = test_end_time - test_start_time
y_pred_proba = y_pred_proba.flatten()

print(f"Testing completed in {test_time:.4f} seconds")

# Step 14: OPTIMIZE THRESHOLD FOR BEST PRECISION-RECALL TRADEOFF
print("\nTesting different classification thresholds:")
print("-" * 70)
print(f"{'Threshold':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'FP':<8} {'FN':<8}")
print("-" * 70)

thresholds_to_test = np.arange(0.3, 0.85, 0.05)
threshold_results = []

for threshold in thresholds_to_test:
    y_pred_temp = (y_pred_proba > threshold).astype(int)

    if sum(y_pred_temp) == 0:  # Skip if no positive predictions
        continue

    prec = precision_score(y_test, y_pred_temp, zero_division=0)
    rec = recall_score(y_test, y_pred_temp, zero_division=0)
    f1_temp = f1_score(y_test, y_pred_temp, zero_division=0)

    cm_temp = confusion_matrix(y_test, y_pred_temp)
    tn_temp, fp_temp, fn_temp, tp_temp = cm_temp.ravel()

    threshold_results.append({
        'threshold': threshold,
        'precision': prec,
        'recall': rec,
        'f1': f1_temp,
        'fp': fp_temp,
        'fn': fn_temp
    })

    print(f"{threshold:<12.2f} {prec:<12.4f} {rec:<12.4f} {f1_temp:<12.4f} {fp_temp:<8} {fn_temp:<8}")

# Select optimal threshold: Maximize F1 while maintaining recall >= 0.75
best_result = None
best_threshold = 0.5

for result in threshold_results:
    if result['recall'] >= 0.75:  # Maintain minimum 75% recall
        if best_result is None or result['f1'] > best_result['f1']:
            best_result = result
            best_threshold = result['threshold']

print("-" * 70)
print(f"\n✓ OPTIMAL THRESHOLD SELECTED: {best_threshold:.2f}")
print(f"  Expected Precision: {best_result['precision']:.4f}")
print(f"  Expected Recall: {best_result['recall']:.4f}")
print(f"  Expected F1-Score: {best_result['f1']:.4f}")
print(f"  False Positives: {best_result['fp']}")
print(f"  False Negatives: {best_result['fn']}")

# Step 15: Use optimal threshold for final predictions
y_pred = (y_pred_proba > best_threshold).astype(int)

# Step 16: Calculate all metrics on TEST SET with optimal threshold
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Calculate PR-AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_proba)
pr_auc = auc(recall_vals, precision_vals)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Step 17: Print all metrics
print("\n" + "="*60)
print("FINAL MODEL EVALUATION METRICS (OPTIMIZED)")
print("="*60)
print(f"Classification Threshold: {best_threshold:.2f} (optimized from 0.50)")
print(f"\nPerformance Metrics:")
print(f"  Accuracy:       {accuracy:.4f}")
print(f"  Precision:      {precision:.4f} 🎯 (IMPROVED)")
print(f"  Recall:         {recall:.4f}")
print(f"  F1-Score:       {f1:.4f} 📈 (IMPROVED)")
print(f"  ROC-AUC Score:  {roc_auc:.4f}")
print(f"  PR-AUC Score:   {pr_auc:.4f}")
print(f"\nTiming:")
print(f"  Train Time:     {train_time:.2f} seconds ({train_time/60:.2f} minutes)")
print(f"  Test Time:      {test_time:.4f} seconds")
print(f"\nConfusion Matrix:")
print(f"  True Negatives:  {tn}")
print(f"  False Positives: {fp} ⬇️ (REDUCED)")
print(f"  False Negatives: {fn}")
print(f"  True Positives:  {tp}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Legitimate', 'Fraud']))

# Step 18: Save all metrics to CSV
metrics_dict = {
    'Model': ['14-Layer CNN (Optimized)'],
    'Epochs': [100],
    'Batch_Size': [2048],
    'Balancing_Method': ['SMOTE 1:1'],
    'Optimization': ['Class Weights + Threshold Tuning'],
    'Classification_Threshold': [best_threshold],
    'Class_Weight_Fraud': [class_weight_dict[1]],
    'Data_Split': ['60% Train / 20% Val / 20% Test'],
    'Validation_Accuracy': [val_accuracy],
    'Test_Accuracy': [accuracy],
    'Test_Precision': [precision],
    'Test_Recall': [recall],
    'Test_F1_Score': [f1],
    'Test_ROC_AUC': [roc_auc],
    'Test_PR_AUC': [pr_auc],
    'Train_Time_seconds': [train_time],
    'Test_Time_seconds': [test_time],
    'Train_Time_minutes': [train_time/60],
    'True_Negatives': [tn],
    'False_Positives': [fp],
    'False_Negatives': [fn],
    'True_Positives': [tp],
    'Total_Test_Samples': [len(y_test)],
    'Test_Fraud_Cases': [sum(y_test)],
    'Test_Legit_Cases': [len(y_test) - sum(y_test)]
}

metrics_df = pd.DataFrame(metrics_dict)
metrics_df.to_csv('14layer_cnn_optimized_metrics.csv', index=False)
print("\n✓ Metrics saved to '14layer_cnn_optimized_metrics.csv'")

# Step 19: Save threshold analysis to CSV
threshold_df = pd.DataFrame(threshold_results)
threshold_df.to_csv('threshold_optimization_analysis.csv', index=False)
print("✓ Threshold analysis saved to 'threshold_optimization_analysis.csv'")

# Step 20: Plot and save ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - 14-Layer CNN (Optimized)', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('14layer_cnn_optimized_roc_curve.png', dpi=300, bbox_inches='tight')
print("✓ ROC curve saved to '14layer_cnn_optimized_roc_curve.png'")
plt.close()

# Step 21: Plot and save Precision-Recall curve
plt.figure(figsize=(10, 6))
plt.plot(recall_vals, precision_vals, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.4f})')
plt.axhline(y=precision, color='red', linestyle='--', lw=1, label=f'Current Precision = {precision:.4f}')
plt.axvline(x=recall, color='green', linestyle='--', lw=1, label=f'Current Recall = {recall:.4f}')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve - 14-Layer CNN (Optimized)', fontsize=14, fontweight='bold')
plt.legend(loc="lower left", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('14layer_cnn_optimized_pr_curve.png', dpi=300, bbox_inches='tight')
print("✓ PR curve saved to '14layer_cnn_optimized_pr_curve.png'")
plt.close()

# Step 22: Plot and save training history
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Model Accuracy (With Class Weights)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Model Loss (With Class Weights)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('14layer_cnn_optimized_training_history.png', dpi=300, bbox_inches='tight')
print("✓ Training history plot saved to '14layer_cnn_optimized_training_history.png'")
plt.close()

# Step 23: Plot threshold analysis
plt.figure(figsize=(12, 6))
threshold_df = pd.DataFrame(threshold_results)

plt.plot(threshold_df['threshold'], threshold_df['precision'], 'o-', label='Precision', linewidth=2, markersize=6)
plt.plot(threshold_df['threshold'], threshold_df['recall'], 's-', label='Recall', linewidth=2, markersize=6)
plt.plot(threshold_df['threshold'], threshold_df['f1'], '^-', label='F1-Score', linewidth=2, markersize=6)
plt.axvline(x=best_threshold, color='red', linestyle='--', linewidth=2, label=f'Optimal Threshold = {best_threshold:.2f}')
plt.xlabel('Classification Threshold', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Threshold Optimization Analysis', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.xlim([0.25, 0.85])
plt.ylim([0.0, 1.05])
plt.tight_layout()
plt.savefig('threshold_optimization_plot.png', dpi=300, bbox_inches='tight')
print("✓ Threshold optimization plot saved to 'threshold_optimization_plot.png'")
plt.close()

# Step 24: Save ROC and PR curve data to CSV
roc_data = pd.DataFrame({
    'False_Positive_Rate': fpr,
    'True_Positive_Rate': tpr
})
roc_data.to_csv('14layer_cnn_optimized_roc_data.csv', index=False)

pr_data = pd.DataFrame({
    'Recall': recall_vals,
    'Precision': precision_vals
})
pr_data.to_csv('14layer_cnn_optimized_pr_data.csv', index=False)

print("✓ ROC curve data saved to '14layer_cnn_optimized_roc_data.csv'")
print("✓ PR curve data saved to '14layer_cnn_optimized_pr_data.csv'")

print("\n" + "="*60)
print("ALL OUTPUTS GENERATED SUCCESSFULLY (OPTIMIZED VERSION)!")
print("="*60)
print("\nOptimization Applied:")
print(f"  ✓ Class Weights: Fraud class weight = {class_weight_dict[1]:.4f}")
print(f"  ✓ Optimal Threshold: {best_threshold:.2f} (tuned from 0.50)")
print(f"\nGenerated Files:")
print("  1. 14layer_cnn_optimized_training_history.csv")
print("  2. 14layer_cnn_optimized_metrics.csv")
print("  3. threshold_optimization_analysis.csv")
print("  4. 14layer_cnn_optimized_roc_curve.png")
print("  5. 14layer_cnn_optimized_pr_curve.png")
print("  6. 14layer_cnn_optimized_training_history.png")
print("  7. threshold_optimization_plot.png")
print("  8. 14layer_cnn_optimized_roc_data.csv")
print("  9. 14layer_cnn_optimized_pr_data.csv")
print("\nExpected Improvement: Precision +10-20%, F1-Score +5-10%")


Original dataset shape: (284807, 31)
Original class distribution:
Class
0    284315
1       492
Name: count, dtype: int64

Before SMOTE:
Training set: 170883 samples - Fraud: 295, Legit: 170588
Validation set: 56962 samples - Fraud: 99, Legit: 56863
Test set: 56962 samples - Fraud: 98, Legit: 56864

After SMOTE (Training set only):
Training set: 341176 samples - Fraud: 170588, Legit: 170588
Validation set: UNCHANGED (original imbalanced distribution)
Test set: UNCHANGED (original imbalanced distribution)

CALCULATING CLASS WEIGHTS FOR PRECISION OPTIMIZATION
Original Class Weights: Class 0 = 1.0000, Class 1 = 1.0000
Adjusted Class Weights: Class 0 = 1.0000, Class 1 = 1.5000
Effect: Model will penalize false positives more heavily


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 29, 32)         │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 29, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 29, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 28, 64)         │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 28, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       114,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │         6,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 132,243 (516.57 KB)

 Trainable params: 132,051 (515.82 KB)

 Non-trainable params: 192 (768.00 B)


STARTING TRAINING WITH CLASS WEIGHTS
Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9021 - loss: 0.2605 - val_accuracy: 0.9968 - val_loss: 0.0162
Epoch 2/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9825 - loss: 0.0546 - val_accuracy: 0.9947 - val_loss: 0.0164
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9897 - loss: 0.0348 - val_accuracy: 0.9948 - val_loss: 0.0168
Epoch 4/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9920 - loss: 0.0268 - val_accuracy: 0.9964 - val_loss: 0.0143
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9936 - loss: 0.0218 - val_accuracy: 0.9973 - val_loss: 0.0126
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9944 - loss: 0.0191 - val_accuracy: 0.9964 - val_loss: 0.0140
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9946 - loss: 0.0181 - val_accuracy: 0.9974 - val_loss: 0.0120
Epoch 8/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - ac

In [5]:
import os
import shutil
import zipfile
from datetime import datetime

# Create directories
print("Creating organized folder structure...")
print("="*60)

folders = {
    'model': 'model/',
    'results': 'results/',
    'plots': 'plots/'
}

for folder_name, folder_path in folders.items():
    os.makedirs(folder_path, exist_ok=True)
    print(f"✓ Created folder: {folder_path}")

# Step 1: Save model weights
print("\n" + "="*60)
print("Saving model weights...")
print("="*60)

model_file = 'model/14layer_cnn_optimized_model.h5'
model.save(model_file)
model_size = os.path.getsize(model_file)
print(f"✓ Model saved: {model_file} ({model_size:,} bytes)")

model_json = model.to_json()
with open('model/14layer_cnn_architecture.json', 'w') as json_file:
    json_file.write(model_json)
print(f"✓ Architecture saved: model/14layer_cnn_architecture.json")

# Step 2: Organize CSV files
print("\n" + "="*60)
print("Moving CSV files to results/...")
print("="*60)

csv_files = [
    '14layer_cnn_optimized_training_history.csv',
    '14layer_cnn_optimized_metrics.csv',
    'threshold_optimization_analysis.csv',
    '14layer_cnn_optimized_roc_data.csv',
    '14layer_cnn_optimized_pr_data.csv'
]

for csv_file in csv_files:
    if os.path.exists(csv_file):
        shutil.move(csv_file, f'results/{csv_file}')
        print(f"✓ Moved: {csv_file} → results/")
    else:
        print(f"✗ Not found: {csv_file}")

# Step 3: Organize plot files
print("\n" + "="*60)
print("Moving plot files to plots/...")
print("="*60)

plot_files = [
    '14layer_cnn_optimized_roc_curve.png',
    '14layer_cnn_optimized_pr_curve.png',
    '14layer_cnn_optimized_training_history.png',
    'threshold_optimization_plot.png'
]

for plot_file in plot_files:
    if os.path.exists(plot_file):
        shutil.move(plot_file, f'plots/{plot_file}')
        print(f"✓ Moved: {plot_file} → plots/")
    else:
        print(f"✗ Not found: {plot_file}")

Creating organized folder structure...
✓ Created folder: model/
✓ Created folder: results/
✓ Created folder: plots/

Saving model weights...
✓ Model saved: model/14layer_cnn_optimized_model.h5 (1,664,436 bytes)
✓ Architecture saved: model/14layer_cnn_architecture.json

Moving CSV files to results/...
✓ Moved: 14layer_cnn_optimized_training_history.csv → results/
✓ Moved: 14layer_cnn_optimized_metrics.csv → results/
✓ Moved: threshold_optimization_analysis.csv → results/
✓ Moved: 14layer_cnn_optimized_roc_data.csv → results/
✓ Moved: 14layer_cnn_optimized_pr_data.csv → results/

Moving plot files to plots/...
✓ Moved: 14layer_cnn_optimized_roc_curve.png → plots/
✓ Moved: 14layer_cnn_optimized_pr_curve.png → plots/
✓ Moved: 14layer_cnn_optimized_training_history.png → plots/
✓ Moved: threshold_optimization_plot.png → plots/


In [6]:
import zipfile
import os
from datetime import datetime

# Create zip filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f'14layer_cnn_results_{timestamp}.zip'

print("Creating zip file...")
print("="*60)

# Create zip file
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    total_files = 0

    # Zip all three folders
    for folder in ['model/', 'results/', 'plots/']:
        if os.path.exists(folder):
            for root, dirs, files in os.walk(folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path)
                    file_size = os.path.getsize(file_path)
                    print(f"✓ Added: {file_path} ({file_size:,} bytes)")
                    total_files += 1
        else:
            print(f"✗ Folder not found: {folder}")

zip_size = os.path.getsize(zip_filename)

print("="*60)
print(f"Zip created: {zip_filename}")
print(f"   Total files: {total_files}")
print(f"   Size: {zip_size:,} bytes ({zip_size/1024/1024:.2f} MB)")

# Download in Colab
try:
    from google.colab import files
    print(f"\n Downloading...")
    files.download(zip_filename)
    print("✓ Download started!")
except ImportError:
    print(f"\n Saved to: {os.path.abspath(zip_filename)}")


Creating zip file...
✓ Added: model/14layer_cnn_architecture.json (9,152 bytes)
✓ Added: model/14layer_cnn_optimized_model.h5 (1,664,436 bytes)
✓ Added: results/threshold_optimization_analysis.csv (869 bytes)
✓ Added: results/14layer_cnn_optimized_training_history.csv (8,286 bytes)
✓ Added: results/14layer_cnn_optimized_roc_data.csv (10,953 bytes)
✓ Added: results/14layer_cnn_optimized_pr_data.csv (1,911,498 bytes)
✓ Added: results/14layer_cnn_optimized_metrics.csv (713 bytes)
✓ Added: plots/14layer_cnn_optimized_roc_curve.png (164,792 bytes)
✓ Added: plots/14layer_cnn_optimized_pr_curve.png (132,859 bytes)
✓ Added: plots/threshold_optimization_plot.png (151,576 bytes)
✓ Added: plots/14layer_cnn_optimized_training_history.png (247,159 bytes)
Zip created: 14layer_cnn_results_20251028_041549.zip
   Total files: 11
   Size: 2,529,856 bytes (2.41 MB)

 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Download started!
